# LLM에 도구 바인딩
- 우리가 만들거나 기존에 존재하는 tool들을 사용하기 위해서는 LLM 모델에 바인딩하는 과정이 필요합니다.
- 두뇌 역할을 하는 LLM에게 사용 할 수 있는 도구들을 쥐어주는 과정이라고 볼 수 있습니다.
- LangChain의 `.bind_tools()` 메소드로 LangChain의 도구 객체를 채팅모델에 전달합니다.

### 도구바인딩 작동 과정
1. 도구정의
2. 사용자 요청분석
3. 도구 호출 결정
4. 실제 도구 실행
5. 결과 전달 및 최종 답변 생성

In [1]:
import os
from dotenv import load_dotenv

# .env 파일에서 환경변수 로드
load_dotenv()

# OpenAI API 환경변수 값 확인
openai_api_key = os.getenv('OPENAI_API_KEY')
print(f"OPENAI_API_KEY가 설정되어 있나요?: {openai_api_key[:10]}...")

OPENAI_API_KEY가 설정되어 있나요?: sk-proj-WS...


## 바인딩할 도구 만들기
- `add_funcition` : 두 수를 더하는 도구
- `sub_function` : 두 수를 빼는 도구
- `divied_function` : 두 수를 나누는 도구

In [2]:
from langchain.tools import tool


@tool
def count_words(text):
    """문자열에 포함된 단어의 수를 세는 도구"""
    return len(text.split())

    
@tool
def sub_function(a: int, b: int) -> int:
    """두 숫자를 뺄셈합니다. 수학적 뺄셈이 필요할 때 사용하세요."""
    return a - b

@tool
def divide_function(a: float, b: float) -> float:
    """두 숫자를 나눕니다. 수학적 나눗셈이 필요할 때 사용하세요. 0으로 나누는 것은 불가능합니다."""
    if b == 0:
        raise ValueError("0으로 나눌 수 없습니다")
    return a / b


tools = [count_words, sub_function, divide_function]

bind_tools()로 LLM에 도구 바인딩

In [3]:
from langchain.chat_models import init_chat_model

# 모델 초기화
llm = init_chat_model("gpt-4o-mini", model_provider="openai", temperature=0.5)

# 도구 바인딩
llm_with_tools = llm.bind_tools(tools)

In [4]:
print(llm_with_tools)

bound=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000025E40E4A1E0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000025E40FE6450>, root_client=<openai.OpenAI object at 0x0000025E3F85AD20>, root_async_client=<openai.AsyncOpenAI object at 0x0000025E40B3AC60>, model_name='gpt-4o-mini', temperature=0.5, model_kwargs={}, openai_api_key=SecretStr('**********')) kwargs={'tools': [{'type': 'function', 'function': {'name': 'count_words', 'description': '문자열에 포함된 단어의 수를 세는 도구', 'parameters': {'properties': {'text': {}}, 'required': ['text'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 'sub_function', 'description': '두 숫자를 뺄셈합니다. 수학적 뺄셈이 필요할 때 사용하세요.', 'parameters': {'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}}, 'required': ['a', 'b'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 'divide_function', 'description': '두 숫자를 나눕니다. 수학적 나눗셈이 필요할 때 사용하세요. 0

실행결과를 확인해보면 tools에 우리가 만든 tool의  
함수명은 `name`, 설명은 `description`, 매개변수는 `parameters` 으로 들어가 있는것을 확인 할 수 있다.

In [5]:
from langchain_core.output_parsers import JsonOutputToolsParser

# 출력 파서 설정
chain = llm_with_tools | JsonOutputToolsParser()

# 실행결과
tool_call_results = chain.invoke("(10-5)/5 을 단계별로 계산해줘")

- type : 도구의 이름
- args : 도구에 전달되는 인자

In [6]:
print(tool_call_results)

[{'args': {'a': 10, 'b': 5}, 'type': 'sub_function'}, {'args': {'a': 5, 'b': 5}, 'type': 'divide_function'}]


In [7]:
def excute_tool_calls(tool_call_results):
    for tool_call in tool_call_results:
        tool_name = tool_call["type"]
        tool_args = tool_call["args"]
        
        # 도구 이름과 일치하는 도구를 찾기
        matching_tool = next((tool for tool in tools if tool.name == tool_name), None)

        # 도구가 존재하는 경우
        if matching_tool:
            # 도구 실행
            result = matching_tool.invoke(tool_args) #
            # 결과 출력
            print(f"도구 {tool_name} 실행 결과: {result}")
        else:
            print(f"도구 {tool_name}을 찾을 수 없습니다.")
            

excute_tool_calls(tool_call_results)

도구 sub_function 실행 결과: 5
도구 divide_function 실행 결과: 1.0
